In [1]:
import gradio as gr
import requests
import json

# Global variable to store API key
groq_api_key = None

def verify_api_key(api_key):
    """Verify if the Groq API key is valid"""
    global groq_api_key

    if not api_key or not api_key.strip():
        return "❌ Please enter an API key", ""

    try:
        # Test the API key by making a request to list models
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        response = requests.get(
            "https://api.groq.com/openai/v1/models",
            headers=headers,
            timeout=10
        )

        if response.status_code == 200:
            groq_api_key = api_key
            return "✅ API Key verified successfully! You can now start chatting.", api_key
        else:
            groq_api_key = None
            return f"❌ Invalid API key. Status code: {response.status_code}", ""

    except Exception as e:
        groq_api_key = None
        return f"❌ Error verifying API key: {str(e)}", ""

def is_photography_related(text):
    """Check if the question is related to photography"""
    photography_keywords = [
        'photo', 'camera', 'lens', 'aperture', 'shutter', 'iso', 'exposure',
        'lighting', 'composition', 'portrait', 'landscape', 'macro', 'bokeh',
        'dslr', 'mirrorless', 'sensor', 'focus', 'zoom', 'flash', 'tripod',
        'filter', 'editing', 'lightroom', 'photoshop', 'raw', 'jpeg', 'megapixel',
        'depth of field', 'histogram', 'white balance', 'focal length', 'f-stop',
        'viewfinder', 'autofocus', 'manual', 'image', 'picture', 'shoot',
        'frame', 'rule of thirds', 'golden hour', 'blue hour', 'long exposure',
        'timelapse', 'hdr', 'panorama', 'selfie', 'photography', 'photographer',
        'canon', 'nikon', 'sony', 'fujifilm', 'olympus', 'panasonic', 'leica'
    ]

    text_lower = text.lower()
    return any(keyword in text_lower for keyword in photography_keywords)

def chat_with_bot(message, history):
    """Handle chat messages"""
    global groq_api_key

    # Check if API key is set
    if not groq_api_key:
        history.append((message, "⚠️ Please verify your API key first using the 'Verify API Key' button above."))
        return history

    # Check if message is empty
    if not message or not message.strip():
        history.append((message, "Please enter a message."))
        return history

    # Check if question is photography-related
    if not is_photography_related(message):
        history.append((message, "🚫 I'm specialized in photography topics only. Please ask me questions related to photography, cameras, techniques, or photo editing!"))
        return history

    try:
        # Prepare messages for API
        messages = [
            {
                "role": "system",
                "content": "You are an expert photography assistant. Only answer questions related to photography, cameras, photo techniques, lighting, composition, editing, and equipment. Provide detailed, helpful, and practical advice. If asked about non-photography topics, politely decline and remind the user you only discuss photography."
            }
        ]

        # Add conversation history
        for user_msg, bot_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": bot_msg})

        # Add current message
        messages.append({"role": "user", "content": message})

        # Call Groq API
        headers = {


            "Authorization": f"Bearer {groq_api_key}",
            "Content-Type": "application/json"
        }

        data = {
            "model": "llama-3.3-70b-versatile",
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 1000
        }

        response = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            headers=headers,
            json=data,
            timeout=30
        )

        if response.status_code == 200:
            result = response.json()
            assistant_message = result['choices'][0]['message']['content']
            history.append((message, assistant_message))
            return history
        else:
            history.append((message, f"❌ Error: Failed to get response from API. Status code: {response.status_code}"))
            return history

    except Exception as e:
        history.append((message, f"❌ Error: {str(e)}"))
        return history

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="Photography Chatbot") as demo:
    gr.Markdown(
        """
        # 📸 Photography Assistant Chatbot
        ### Powered by Groq API

        This chatbot specializes in photography topics only. Ask questions about cameras, techniques, lighting, composition, editing, and more!
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            api_key_input = gr.Textbox(
                label="Groq API Key",
                placeholder="Enter your Groq API key here...",
                type="password",
                info="Get your API key from https://console.groq.com"
            )
        with gr.Column(scale=1):
            verify_btn = gr.Button("🔑 Verify API Key", variant="primary", size="lg")

    api_status = gr.Textbox(
        label="API Key Status",
        interactive=False,
        show_label=True
    )

    gr.Markdown("---")

    chatbot = gr.Chatbot(
        label="Chat with Photography Assistant",
        height=500,
        show_label=True,
        avatar_images=(None, "🎥")
    )

    with gr.Row():
        msg = gr.Textbox(
            label="Your Question",
            placeholder="Ask me anything about photography...",
            scale=4,
            lines=2
        )
        send_btn = gr.Button("📤 Send", variant="primary", scale=1)

    gr.Markdown(
        """
        ### 💡 Example Questions:
        - What are the best camera settings for portrait photography?
        - How do I achieve a blurred background effect?
        - What's the difference between aperture and shutter speed?
        - How can I improve my landscape photography?
        - What equipment do I need to start photography?
        """
    )

    # Event handlers
    verify_btn.click(
        fn=verify_api_key,
        inputs=[api_key_input],
        outputs=[api_status, api_key_input]
    )

    msg.submit(
        fn=chat_with_bot,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )

    send_btn.click(
        fn=chat_with_bot,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=False, server_name="0.0.0.0")

/tmp/ipython-input-2889737772.py:125: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="Photography Chatbot") as demo:
/tmp/ipython-input-2889737772.py:154: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2889737772.py:154: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>